In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize import minimize


C is the list (of lists) of calibration tuples $(p(t_i), \tilde{q}(t_j), t_l)$, where $t_l$ is the joint measurement time $t_l = (t_i + t_j) / 2$. $p(t_i)$ is the measured value. $\tilde{q}(t_j)$ is the calibration provided by another sensor.
I've made C a list of lists, one for each sensor.

We adjust $\gamma_1(T_m)$ to minimise the least squares error between $c(\gamma(T_m),p(t_i))$ and $\tilde{q}(t_j)$.

In [5]:
X = []
Y = []
X.append([ 0,0,1]); Y.append([15,15])
X.append([ 1,0,1]); Y.append([12,12])
X.append([ 2,0,1]); Y.append([10,10])
X.append([33,1,2]); Y.append([10,14])
X.append([34,1,2]); Y.append([20,24])
X.append([35,1,2]); Y.append([30,34])
X = np.array(X)
Y = np.array(Y)
gamma = [[1,0],[0,0],[0,0]]
C = [[],[],[]]
ages = np.array([0,1e6,1e6])
lasttime = 0
Tm = 36

def w(deltat):
    return 1/(1+deltat)
def cal(params,pollution):
    return params[0]*pollution+params[1]
def geterror(cs,g):
    s = 0
    for c in cs:
        s+=w(Tm-c[2])*(cal(g,c[0])-c[1])**2
    return s

for x,y in zip(X,Y):
    timestepsize = x[0]-lasttime
    lasttime = x[0]
    ages += timestepsize
    ages[0] = 0 #reference
    t1 = ages[x[1]]
    t2 = ages[x[2]]
    #if (t1==0) & (t2==0): pol = y[0]
    pol = (t1*cal(gamma[1],y[1])+t2*cal(gamma[0],y[0]))/(t1+t2)
    tl = x[0]
    C[x[1]].append([y[0],pol,tl,t1])
    C[x[2]].append([y[1],pol,tl,t2])
    #if t1==np.inf: 
    #    avgage = t2
    #else:
    #    if t2==np.inf: 
    #        avgage = t1
    #    else:
    #        avgage = (t1+t2)/2
    
    print("-----")
    print(ages)
    for i,(cs,g) in enumerate(zip(C,gamma)):
        print(cs)
        res = minimize(lambda newg: geterror(cs,newg), g)
        print(res.x)
        gamma[i] = res.x
    print("========================")
    ages[x[1]] = avgage
    ages[x[2]] = avgage

-----
[      0. 1000000. 1000000.]
[[15, 15.0, 0, 0.0]]
[1. 0.]
[[15, 15.0, 0, 1000000.0]]
[0.99557521 0.06637169]
[]
[0. 0.]
-----
[      0.  500001. 1000001.]
[[15, 15.0, 0, 0.0], [12, 12.0, 1, 0.0]]
[1. 0.]
[[15, 15.0, 0, 1000000.0], [12, 12.0, 1, 500001.0]]
[0.99971932 0.00382973]
[]
[0. 0.]
-----
[      0.   250001.5 1000002. ]
[[15, 15.0, 0, 0.0], [12, 12.0, 1, 0.0], [10, 10.0, 2, 0.0]]
[1. 0.]
[[15, 15.0, 0, 1000000.0], [12, 12.0, 1, 500001.0], [10, 10.0, 2, 250001.5]]
[9.99999747e-01 3.10289734e-06]
[]
[0. 0.]
-----
[      0.    125031.75 1000033.  ]
[[15, 15.0, 0, 0.0], [12, 12.0, 1, 0.0], [10, 10.0, 2, 0.0]]
[1. 0.]
[[15, 15.0, 0, 1000000.0], [12, 12.0, 1, 500001.0], [10, 10.0, 2, 250001.5], [10, 10.444531699017215, 33, 125031.75]]
[0.90620187 1.32498952]
[[14, 10.444531699017215, 33, 1000033.0]]
[0.74225097 0.05301801]
-----
[     0.    562533.375 562533.375]
[[15, 15.0, 0, 0.0], [12, 12.0, 1, 0.0], [10, 10.0, 2, 0.0]]
[1. 0.]
[[15, 15.0, 0, 1000000.0], [12, 12.0, 1, 500001.

In [6]:
for c in C: print(c,'\n')

[[15, 15.0, 0, 0.0], [12, 12.0, 1, 0.0], [10, 10.0, 2, 0.0]] 

[[15, 15.0, 0, 1000000.0], [12, 12.0, 1, 500001.0], [10, 10.0, 2, 250001.5], [10, 10.444531699017215, 33, 125031.75], [20, 21.536917230668692, 34, 562533.375], [30, 33.571421920469064, 35, 562534.375]] 

[[14, 10.444531699017215, 33, 1000033.0], [24, 21.536917230668692, 34, 562533.375], [34, 33.571421920469064, 35, 562534.375]] 



In [109]:
gamma

[array([1., 0.]), array([0., 0.]), array([0., 0.])]

In [96]:
ages

array([-102., -102., -102.])

why is the the age of a calibration 
> "the sum of (i) the time passed since a sensor’s most recent calibration T m and (ii) the average calibration age of the sensor’s calibration parents"

surely if one of the parents is a reference then... also this isn't very clear. What are its parents? The two sensors?

In [74]:

#C = [[(10,14.0,33),(20,24,34),(30,34,35),(15,15,0)]]


for cs,g in zip(C,gamma):
    geterror(cs,g)

In [37]:
from scipy.optimize import minimize
res = minimize(lambda newg: geterror(cs,newg), g)
newg = res.x
newg

array([1.01091164, 3.66116439])